Pyspark -ML

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('missimg').getOrCreate()

In [3]:
spark

In [4]:
df=spark.read.csv(r'D:\Naresh_it_praksah_senapathi\sept_DL\Spark\sample1.csv',header=True,inferSchema=True)

In [5]:
df

DataFrame[Name: string, age: int, Experience: int, Salary: int]

In [6]:
df.show()

+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|    jack| 31|        10| 30000|
|    alex| 30|         8| 25000|
|caroline| 29|         4| 20000|
|    paul| 24|         3| 20000|
|  sandra| 21|         1| 15000|
|casandra| 23|         2| 18000|
+--------+---+----------+------+



In [8]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
#age and experience are independant features

In [11]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age','Experience'],outputCol='Independent features')

In [12]:
output=featureassembler.transform(df)

In [13]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|age|Experience|Salary|Independent features|
+--------+---+----------+------+--------------------+
|    jack| 31|        10| 30000|         [31.0,10.0]|
|    alex| 30|         8| 25000|          [30.0,8.0]|
|caroline| 29|         4| 20000|          [29.0,4.0]|
|    paul| 24|         3| 20000|          [24.0,3.0]|
|  sandra| 21|         1| 15000|          [21.0,1.0]|
|casandra| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [14]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent features']

In [15]:
finalized_data=output.select("Independent features","Salary")

In [16]:
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression
#train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent features",labelCol="Salary")
regressor=regressor.fit(train_data)

In [19]:
regressor.coefficients

DenseVector([-2087.7193, 3561.4035])

In [20]:
#intercepts
regressor.intercept

59140.35087715416

In [21]:
#prediction
pred_results=regressor.evaluate(test_data)

In [23]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|18859.649122805153|
|          [29.0,4.0]| 20000|12842.105263165002|
+--------------------+------+------------------+



In [24]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(5508.7719298200755, 33066174.20739037)